In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
# Question 1: Execute spark.version.

spark = SparkSession.\
        builder.\
        appName('week_6').\
        master('local[2]').\
        getOrCreate()

spark.version

#### Answer: 3.3.2

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


26/02/21 05:16:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


'3.3.2'

In [3]:
# Question 2: Yellow November 2025
# Read the November 2025 Yellow into a Spark Dataframe.
# Repartition the Dataframe to 4 partitions and save it to parquet.
# What is the average size of the Parquet (ending with .parquet extension) Files that were created (in MB)?
# Select the answer which most closely matches.

!curl -Lf https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-11.parquet -o yellow.parquet

taxi = spark.read.parquet('yellow.parquet', header = True, inferSchema = True)
taxi = taxi.repartition(4)
taxi.write.parquet('yellow', 'overwrite')
!ls -la --block-size M yellow/
taxi.createOrReplaceTempView('taxi')

#### Answer: 28 MB

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 67.8M  100 67.8M    0     0  62.0M      0  0:00:01  0:00:01 --:--:-- 62.0M


total 111M
drwxr-xr-x 2 ajay ajay  1M Feb 21 05:17 .
drwxrwxr-x 4 ajay ajay  1M Feb 21 05:17 ..
-rw-r--r-- 1 ajay ajay  1M Feb 21 05:17 ._SUCCESS.crc
-rw-r--r-- 1 ajay ajay  1M Feb 21 05:17 .part-00000-dcfdb919-0d54-41de-bc89-9b67ae7f85de-c000.snappy.parquet.crc
-rw-r--r-- 1 ajay ajay  1M Feb 21 05:17 .part-00001-dcfdb919-0d54-41de-bc89-9b67ae7f85de-c000.snappy.parquet.crc
-rw-r--r-- 1 ajay ajay  1M Feb 21 05:17 .part-00002-dcfdb919-0d54-41de-bc89-9b67ae7f85de-c000.snappy.parquet.crc
-rw-r--r-- 1 ajay ajay  1M Feb 21 05:17 .part-00003-dcfdb919-0d54-41de-bc89-9b67ae7f85de-c000.snappy.parquet.crc
-rw-r--r-- 1 ajay ajay  0M Feb 21 05:17 _SUCCESS
-rw-r--r-- 1 ajay ajay 28M Feb 21 05:17 part-00000-dcfdb919-0d54-41de-bc89-9b67ae7f85de-c000.snappy.parquet
-rw-r--r-- 1 ajay ajay 28M Feb 21 05:17 part-00001-dcfdb919-0d54-41de-bc89-9b67ae7f85de-c000.snappy.parquet
-rw-r--r-- 1 ajay ajay 28M Feb 21 05:17 part-00002-dcfdb919-0d54-41de-bc89-9b67ae7f85de-c000.snappy.parquet
-rw-r--r-- 1 ajay ajay 28

In [4]:
# Question 3: Count records
# How many taxi trips were there on the 15th of November?
# Consider only trips that started on the 15th of November.

spark.sql('''
            select 
                count(*) as records_count
            from taxi 
            where 
                date(tpep_pickup_datetime) = '2025-11-15'
          ''').show()

#### Answer: 162,604

[Stage 4:>                                                          (0 + 2) / 2]

+-------------+
|records_count|
+-------------+
|       162604|
+-------------+



In [5]:
# Question 4: Longest trip
# What is the length of the longest trip in the dataset in hours?

spark.sql('''
            select
                round(
                    max(unix_timestamp(tpep_dropoff_datetime) - unix_timestamp(tpep_pickup_datetime)
                        ) / 3600
                    , 2) as trip_duration
            from taxi
            ''').show()

#### Answer: 90.65

[Stage 12:=============================>                            (2 + 2) / 4]

+-------------+
|trip_duration|
+-------------+
|        90.65|
+-------------+



In [6]:
# Question 5: User Interface
# Spark’s User Interface which shows the application's dashboard runs on which local port?

spark._activeSession

#### Answer: 4040

In [7]:
# Question 6: Least frequent pickup location zone
# Using the zone lookup data and the Yellow November 2025 data, what is the name of the LEAST frequent pickup location Zone?

!curl -Lf https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv -o zone.csv
zone = spark.read.csv('zone.csv', header = True, inferSchema = True)
zone.createOrReplaceTempView('zone')

spark.sql('''
            select
                z.Zone,
                count(*) as num_trips
            from taxi t
            join zone z on
                z.LocationId = t.PULocationID
            group by 1
            order by 2
            ''').head(5)

#### Answer: Arden Heights

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 12331  100 12331    0     0  74499      0 --:--:-- --:--:-- --:--:-- 74733


[Row(Zone='Arden Heights', num_trips=1),
 Row(Zone="Eltingville/Annadale/Prince's Bay", num_trips=1),
 Row(Zone="Governor's Island/Ellis Island/Liberty Island", num_trips=1),
 Row(Zone='Port Richmond', num_trips=3),
 Row(Zone='Green-Wood Cemetery', num_trips=4)]

In [8]:
!rm -rf yellow yellow.parquet zone.csv